In [1]:
import sys

sys.path.append('D:\PythonProjects\\bk-monitor\\venv2\share\scripts')
import hello

当前运行环境缺少部分需要的环境变量，可能导致项目运行报错，请确认以下环境变量已经配置并赋值: BK_PAAS2_URL
注意，当前版本为社区版环境，请确认 BKPAAS_MAJOR_VERSION 已经配置并正确赋值


In [15]:
from mock import patch, MagicMock
from pprint import pprint
from bkmonitor.documents import alert
from bkmonitor import documents
from itertools import cycle
from bkmonitor import models
from copy import deepcopy


def func():
    dimensions = ["name", "age", "sex"]
    dic = {
        "name": "张三",
        "age": 18,
        "sex": "male"
    }

    class Inner:
        def __init__(self, key, value):
            self.dic = {
                "key": key,
                "value": dic[key],
                "dispaly_key": key,
                "display_value": dic[key]
            }

        def to_dict(self):
            return self.dic

        def __getitem__(self, key):
            return self.dic[key]

        def get(self, key, default):
            return self.dic.get(key, default)

    for key in cycle(dimensions):
        yield Inner(key, dic[key])


class Inner:
    id = 1


inner = Inner()


def create(*args, **kwargs):
    for key, value in kwargs.items():
        setattr(inner, key, value)
    return inner


alert_doc = alert.AlertDocument = documents.AlertDocument = MagicMock(name="AlertDocument")
mock_alert = MagicMock(name="alert")
mock_alert.id = 1
generator = func()
mock_alert.dimensions = [next(generator), next(generator), next(generator)]
alert_doc.get = MagicMock(return_value=mock_alert)

models.Shield = MagicMock(name="Shield")
models.Shield.objects = MagicMock(name="Shield.objects")
models.Shield.objects.create = MagicMock(name="Shield.objects.create", side_effect=create)

from weixin.event.resources import QuickShield

QuickShield.has_biz_permission = MagicMock(return_value=True)



In [20]:
# 测试删除“sex” 维度
params = {
    "type": "event",
    "event_id": 1,
    "bk_biz_id": 2,
    "end_time": "2023-10-05 14:30:00",
    "description": "测试",
    # 没有sex维度，意思就是删除sex维度
    "dimension_keys": ["name", "age"],
    "category": "event",
}

QuickShield().request(params)
pprint(inner.dimension_config["_dimensions"])

'name(张三) - age(18)'


In [21]:
# 测试没有传递dimension_keys参数的情况
new_params = deepcopy(params)
new_params.pop("dimension_keys")
QuickShield().request(new_params)
pprint(inner.dimension_config["_dimensions"])

'name(张三) - age(18) - sex(male)'


In [22]:
# 测试dimension_keys参数为空列表的情况
new_params = deepcopy(params)
new_params["dimension_keys"] = []
QuickShield().request(new_params)
pprint(inner.dimension_config["_dimensions"])

''
